# Cryptocurrency trading bot.

## Install if needed.

In [1]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

## API key management.

In [2]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
client

## Get all available pairs for information needed to trade.

In [3]:
from cryptocurrency.exchange import Cryptocurrency_exchange
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
exchange_info

,symbol,base_asset,base_asset_precision,quote_asset,quote_precision,quote_asset_precision,base_asset_commission,quote_commission_precision,allow_trailing_stop,cancel_replace_allowed,min_price,max_price,tick_size,step_size,multiplier_up,multiplier_down
0,ETHBTC,ETH,8,BTC,8,8,8,8,True,True,1.000000e-06,922327.0,1.000000e-06,0.0001,5.0,0.2
1,LTCBTC,LTC,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
2,BNBBTC,BNB,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
3,NEOBTC,NEO,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0100,5.0,0.2
4,QTUMETH,QTUM,8,ETH,8,8,8,8,True,True,1.000000e-06,1000.0,1.000000e-06,0.1000,5.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,VITEBUSD,VITE,8,BUSD,8,8,8,8,True,True,1.000000e-05,1000.0,1.000000e-05,0.1000,5.0,0.2
1411,HOOKBTC,HOOK,8,BTC,8,8,8,8,True,True,1.000000e-08,1.0,1.000000e-08,0.1000,5.0,0.2
1412,HOOKUSDT,HOOK,8,USDT,8,8,8,8,True,True,1.000000e-04,1000.0,1.000000e-04,0.1000,5.0,0.2
1413,HOOKBUSD,HOOK,8,BUSD,8,8,8,8,True,True,1.000000e-04,1000.0,1.000000e-04,0.1000,5.0,0.2


## Get conversion table for price conversions.

In [4]:
from_asset = 'BDOT'
to_asset = 'BETH'

from cryptocurrency.conversion import get_shortest_pair_path_between_assets

shortest_pair_path_between_assets = \
    get_shortest_pair_path_between_assets(from_asset, to_asset, 
                                          exchange_info, 
                                          priority='fees')

shortest_pair_path_between_assets

[('BDOT', 'DOT'), ('DOT', 'BUSD'), ('BETH', 'BUSD')]

In [5]:
from cryptocurrency.conversion_table import get_conversion_table
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table

,symbol,price_change,price_change_percent,weighted_average_price,close_shifted,close,last_volume,bid_price,bid_volume,ask_price,...,rolling_quote_volume,open_time,first_ID,last_ID,count,base_asset,quote_asset,rolling_base_quote_volume,bid_ask_percent_change,bid_ask_volume_percent_change
close_time,,,,,,,,,,,,,,,,,,,,,
2022-12-08 22:00:03.235,CVCBTC,-0.00000004,-0.702,0.00000567,0.000006,0.000006,122.00000000,0.00000,0.000,0.0000,...,7.477116e+00,1670468403235,9726973,9727929,957,CVC,BTC,1.321045e+06,NaN,NaN
2022-12-08 22:00:15.228,CVCUSDT,0.00260000,2.731,0.09684944,0.095300,0.097800,4059.00000000,0.00000,0.000,0.0000,...,7.213074e+05,1670468415228,28926988,28930852,3865,CVC,USDT,7.375332e+06,NaN,NaN
2022-12-08 22:00:20.670,ROSEBNB,-0.00000180,-1.160,0.00015905,0.000154,0.000153,900.00000000,0.00000,0.000,0.0000,...,1.194225e+03,1670468420670,230065,231163,1099,ROSE,BNB,7.785037e+06,NaN,NaN
2022-12-08 22:00:26.231,SLPBNB,0.00000006,0.683,0.00000909,0.000009,0.000009,1128668.00000000,0.00000,0.000,0.0000,...,7.279490e+02,1670468426231,1140633,1141515,883,SLP,BNB,8.234717e+07,NaN,NaN
2022-12-09 13:12:36.308,BUSDVAI,-0.00200000,-0.194,1.03221196,1.031000,1.031000,29.00000000,1.03100,942.700,1.0340,...,7.597080e+01,1670523156308,602533,602534,2,BUSD,VAI,7.368652e+01,0.290135,15.939603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-09 18:40:00.316,ANTUSDT,0.10800000,5.044,2.23994654,2.139000,2.249000,56.70000000,2.24900,96.300,2.2500,...,5.133480e+06,1670542800316,40269223,40301842,32620,ANT,USDT,2.282561e+06,0.044444,5.333703
2022-12-09 18:40:00.317,DASHETH,0.00158000,4.490,0.03768279,0.035360,0.036770,4.31200000,0.03682,42.720,0.0369,...,1.368344e+02,1670542800317,3822047,3822488,442,DASH,ETH,3.721360e+03,0.216802,80.002996
2022-12-09 18:40:00.317,KAVABUSD,0.00200000,0.229,0.87691910,0.874000,0.877000,21.50000000,0.87700,3520.900,0.8780,...,8.616118e+05,1670542800317,2678374,2684748,6375,KAVA,BUSD,9.824536e+05,0.113895,93.758155


## Pre-screen buyable base assets in pairs.

In [6]:
from cryptocurrency.conversion_table import get_new_tickers
tickers_list = get_new_tickers(conversion_table=conversion_table)
len(tickers_list)

1415

## Locally (saving time) convert price from an asset to another using existing pairs.

In [7]:
from cryptocurrency.trade import select_asset_with_biggest_wallet
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info, 
                                     as_pair=True)
from_asset, converted_quantity, quantity

('ALPINE', '39.2442', 14.4005)

## Trade.

In [8]:
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True, dump_raw=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
#conversion_table

In [9]:
to_asset = 'BTC'
priority = 'fees'

from cryptocurrency.trade import trade
import pandas as pd

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                exchange_info=exchange_info, priority=priority)
from_asset = to_asset
request

[('ALPINE', 'BTC')]


{'symbol': 'ALPINEBTC',
 'orderId': 55016338,
 'orderListId': -1,
 'clientOrderId': 'zM9n8hOPM5DrfZESbsSxSp',
 'transactTime': 1670629206260,
 'price': '0.00000000',
 'origQty': '14.40000000',
 'executedQty': '14.40000000',
 'cummulativeQuoteQty': '0.00228797',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '0.00015889',
   'qty': '11.46000000',
   'commission': '0.00000182',
   'commissionAsset': 'BTC',
   'tradeId': 3663409},
  {'price': '0.00015888',
   'qty': '2.94000000',
   'commission': '0.00000047',
   'commissionAsset': 'BTC',
   'tradeId': 3663410}]}

## Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [ ]:
priority = 'fees'
sell_asset = 'BUSD'
take_profit = None
stop_loss = None
bought_asset = None
asset_to_delete = None
crypto_input_log_screened_15s = '~/workspace/crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened_15s = '~/workspace/crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = '~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'
crypto_output_log_screened_1h = '~/workspace/crypto_logs/crypto_output_log_1h_screened.txt'
crypto_output_log_screened_1d = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'
crypto_input_log_screened = crypto_input_log_screened_15s
crypto_output_log_screened = crypto_output_log_screened_1d

keys_file = 'contabo_keys.txt'
with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

#ssh.connect('154.12.239.24', username='sam', port=22, password=password)
#ssh.connect('154.53.55.232', username='sam', port=22, password=password)
ssh.connect('178.238.230.58', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    return pd.read_csv(StringIO(str(ssh_stdout.read().decode('utf-8'))))

def choose_to_asset(from_asset, to_asset, latest_asset, crypto_output_log_screened=crypto_output_log_screened):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset, price_when_bought, asset_to_delete):
    conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            pair_bought = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            asset_to_delete = from_asset
        from_asset = to_asset
    return from_asset, to_asset, price_when_bought, asset_to_delete

latest_asset = from_asset
price_when_bought = quantity
while True:
    latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                             crypto_output_log_screened=crypto_output_log_screened)
    if from_asset != to_asset:
        from_asset, to_asset, price_when_bought, asset_to_delete = \
            trade_conditionally(client=client, exchange_info=exchange_info, to_asset=to_asset, 
                                price_when_bought=price_when_bought, asset_to_delete=asset_to_delete)
    time.sleep(2)

ssh.close()

.[('BTC', 'BUSD'), ('FTT', 'BUSD')]
..................................................[('FTT', 'BUSD')]
.[('FTT', 'BUSD')]
.....................[('FTT', 'BUSD')]


In [ ]:
priority = 'fees'
sell_asset = 'BUSD'
take_profit = 10.0
stop_loss = None
bought_asset = None
asset_to_delete = None
crypto_input_log_screened_15s = '~/workspace/crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened_15s = '~/workspace/crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = '~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'
crypto_output_log_screened_1h = '~/workspace/crypto_logs/crypto_output_log_1h_screened.txt'
crypto_output_log_screened_1d = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'
crypto_input_log_screened = crypto_input_log_screened_15s
crypto_output_log_screened = crypto_output_log_screened_1d

keys_file = 'contabo_keys.txt'
with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    return pd.read_csv(StringIO(str(ssh_stdout.read().decode('utf-8'))))

def choose_to_asset(from_asset, to_asset, latest_asset, crypto_output_log_screened=crypto_output_log_screened):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset, price_when_bought, asset_to_delete):
    conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            pair_bought = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            asset_to_delete = from_asset
        from_asset = to_asset
    return from_asset, to_asset, price_when_bought, asset_to_delete

latest_asset = from_asset
price_when_bought = quantity
while True:
    try:
        if asset_to_delete is None:
            latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                                     crypto_output_log_screened=crypto_output_log_screened)
            if from_asset != to_asset:
                from_asset, to_asset, price_when_bought, asset_to_delete = \
                    trade_conditionally(client=client, exchange_info=exchange_info, to_asset=to_asset, 
                                        price_when_bought=price_when_bought, asset_to_delete=asset_to_delete)
            elif bought_asset is not None:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['close'].iat[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
crypto_input_log_screened_15s = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened_15s = 'crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = 'crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = 'crypto_logs/crypto_output_log_30min_screened.txt'
crypto_output_log_screened_1h = 'crypto_logs/crypto_output_log_1h_screened.txt'
crypto_output_log_screened_1d = 'crypto_logs/crypto_output_log_1d_screened.txt'
crypto_input_log_screened = crypto_input_log_screened_15s
crypto_output_log_screened = crypto_output_log_screened_1d

keys_file = 'contabo_keys.txt'
with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_1d):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    return pd.read_csv(StringIO(str(ssh_stdout.read().decode('utf-8'))))

while True:
    df = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)

    if df.shape[0] < 1:
        print('None')
    else:
        print(df)

    time.sleep(2)

ssh.close()